In [8]:
import re, subprocess, pyodbc , os
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd 

In [52]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT GROUPDESC, QPK, QGROUP, QuestionTypeCode, QTTable,  ROW_NUMBER() OVER(ORDER BY IDTEXT ASC) -1 AS Row 
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%EINS%' AND QuestionTypeCode LIKE '%NCEinstein%'
ORDER BY IDTEXT ASC 
"""  
df = pd.read_sql(query,con=conn)  
conn.close()
df.GROUPDESC = df.GROUPDESC.apply(lambda s: re.sub('Section \d:','',s))
df 

,GROUPDESC,QPK,QGROUP,QuestionTypeCode,QTTable,Row
0,Public IP Address Specification,20850,2110,NCEinsteinPublicIP,EinsteinPublicIP,0
1,Unannounced IP Address Specification,20861,2111,NCEinsteinUnannounced,EinsteinUnannounced,1
2,Test Access Point (TAP) Locations,20881,2113,NCEinsteinTaps,EinsteinTaps,2
3,Virtual LANs (VLANs) Specification,20891,2114,NCEinsteinVLAN,EinsteinVLAN,3
4,Unmonitored Internet Traffic,20900,2115,NCEinsteinUnmonitoredTraffic,EinsteinUnmonitoredTraffic,4
5,Network Address Translation (NAT) Specification,20940,2120,NCEinsteinNATInfo,EinsteinNAT,5
6,Border Gateway Protocol (BGP) Specification,20950,2121,NCEinsteinBGP,EinsteinBGP,6
7,Network Diagrams,20960,2122,NCEinsteinArtifactGrid,None,7


In [ ]:
s=''
for i, r in df.iterrows(): 
    s=f"IF @PK_QuestionGroup = {r['QGROUP']} -- {r['QTTable']}\n "
    s=s+f'BEGIN\n'
    s=s+f"     SET @ANS_1_1 = (SELECT Answer FROM vwMetricsAnswers WHERE PK_OrgSubmission = @PK_OrgSubmission AND IdText = '1.1' AND PK_QuestionGroup = {r['QGROUP']})\n"
    s=s+f"     SET @GRIDROWS = (SELECT COUNT(*) FROM {r['QTTable']} WHERE PK_OrgSubmission = @PK_OrgSubmission)\n"
    s=s+f"      \n"
    s=s+f'END\n'
    print(s)

In [53]:
l = [] 
for i in range(2,9):  
    dff = pd.read_csv(fr'c:\temp\S{i}.csv')
    dff['SECTION'] = i 
    l.append(dff)
dfs=pd.concat(l)
dfs.columns = [c.replace(' ','') for c in dfs.columns]
dfs.columns 
#dfs

Index(['ColumnHeadings', 'DataEntryValidations', 'EnumeratedList',
       'AdditionalHelpText', 'SECTION', 'Feature'],
      dtype='object')

,ColumnHeadings,DataEntryValidations,EnumeratedList,AdditionalHelpText,SECTION,Feature
0,Starting IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,First address of the contiguous IP range in IP...,2,NaN
1,Ending IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,Last address of the contiguous IP range that c...,2,NaN
2,Name/Purpose,Optional Entry\n\nValidation\nSubstitute with ...,NaN,A short description of the address block that ...,2,NaN
3,Visible to EINSTEIN?,Mandatory Entry\n\nValidation\nMust be one of ...,Yes\nNo\nUnknown,Is this address block presented to EINSTEIN fo...,2,NaN
4,External Organization,Optional Entry\n\nValidation\nSubstitute with ...,NaN,If this address range is associated with an ex...,2,NaN
0,Network Aggregation TAP Identifier,Mandatory Entry\n\nValidation\nThe field is re...,NaN,Your organization’s identifier for the EINSTEI...,3,NaN
1,Site Identifier,Mandatory Entry\n\nValidation\nThe field is re...,NaN,The identifier of the data center or facility ...,3,NaN
2,Logical Location,Mandatory Entry\n\nValidation\nOne of the enum...,Outside Perimeter Router\nOutside Perimeter Fi...,Select the zone from the figure. If the locati...,3,NaN
3,Other Logical Location,Optional Entry\n\nValidation\n- If the “Logica...,NaN,"If you selected “Other” for Logical Location, ...",3,NaN
4,EINSTEIN System,Mandatory Entry\n\nValidation\nOne of the enum...,E1\nE2\nE1 & E2,Indicate what type of EINSTEIN device this TAP...,3,NaN


In [75]:
dirr= r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\EINSTEIN\2021' 
templates = []
pattern = 'EINSTEIN_0.aspx\.vb$' 
files = [f for f in os.scandir(dirr) if re.search(pattern, f.path )]    
for file in files:  
    with open(file.path, encoding="utf-8", errors='ignore', mode='r' ) as f: 
        templates.append({'d':f'{dirr}\\{file.name}', 'c':f.read() })   
for t in templates:
    print( t['d'] )

D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\EINSTEIN\2021\2021_A_EINSTEIN_0.aspx.vb


In [79]:
for i, r in df[1:7].iterrows(): 
    for t in templates:   
        dd = t['d'].replace('_0',f'_{i+0}') 
        dd = dd.replace('NCEinsteinPublicIP',f"{r['QuestionTypeCode']}" )
        cc = t['c'].replace('_0',f'_{i+0}') 
        cc = cc.replace('NCEinsteinPublicIP',f'EinsteinPublicIP') 
        cc = cc.replace('EinsteinPublicIP',r['QuestionTypeCode']) 
        cc = cc.replace('SectionCaption="Public IP Space"', f"SectionCaption=\"{r['GROUPDESC']}\"" )
        cc = cc.replace('2110',str(r['QGROUP'])) 
        cc = cc.replace('20850',str(r['QPK'])) 
        cc = cc.replace('20851',str(r['QPK']+1))
        cc = cc.replace('20852',str(r['QPK']+2)) 
        cc = cc.replace('1.2 Explanation',str(i+1) + '.2 Explanation') 
        
        #print( cc )
        print(cc)
        #with open(dd, 'w', encoding="utf-8") as writer: 
        #    writer.write(cc)

﻿Imports CyberBalance.VB.Core
Imports CyberBalance.VB.Web.UI
Public Class _2021_A_EINSTEIN_1
    Inherits FismaFormBasePage2

    Protected Sub Page_Init(sender As Object, e As EventArgs) Handles Me.Init
        bl_Errors.Items.Clear()
        Me.ShowRow2 = "none"
    End Sub
    Protected Sub Page_PreRender(sender As Object, e As EventArgs) Handles Me.PreRender
        If Not String.IsNullOrEmpty(CBYesNo_1.Value) Then
            hidShowGrid.Value = CBYesNo_1.Value
            Me.ShowRow2 = IIf(CBYesNo_1.Value <> "Y", String.Empty, "none")
        End If
        bl_Errors.Visible = bl_Errors.Items.Count > 0
        ScriptManager.RegisterClientScriptInclude(Me, Me.GetType(), $"EinsteinSection", Page.ResolveClientUrl("~/scripts/EinsteinSection.js"))
    End Sub
    Protected Overrides Function FormValid(Optional ByVal bApprovalSubmit As Boolean = False) As Boolean
        If Not String.IsNullOrEmpty(CBYesNo_1.Value) Then
            If CBYesNo_1.Value <> "Y" And String.IsNullOrEmpty(CBT

In [47]:
dirr= r"D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls"
templates = []
ffilter = 'NCEinsteinPublicIP' 
files = [f for f in os.scandir(dirr) if ffilter in f.path ]    
for file in files:  
    with open(file.path, encoding="utf-8", errors='ignore', mode='r' ) as f: 
        templates.append({'d':f'{dirr}\\{file.name}', 'c':f.read() })   

In [61]:
# templates[0]

In [56]:
for i, r in df[4:5].iterrows(): 
    for t in templates: 
        dd = t['d']
        cc = t['c']
        dd = dd.replace('NCEinsteinPublicIP',f"{r['QuestionTypeCode']}" )  
        cc = cc.replace('NCEinsteinPublicIP',f"{r['QuestionTypeCode']}")  
        cc = cc.replace('EinsteinPublicIP',f"{r['QTTable']}") 
        print( dd )
        with open(dd, 'w', encoding="utf-8") as writer:
            writer.write(cc)


D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls\NCEinsteinUnmonitoredTraffic.ascx
D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls\NCEinsteinUnmonitoredTraffic.ascx.designer.vb
D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\CustomControls\NCEinsteinUnmonitoredTraffic.ascx.vb
